## Stress en weersomstandigheden
### Set up


In [2]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
from IPython.core.display import display
from IPython.core.interactiveshell import InteractiveShell



In deze studie vragen we ons af welke invloed het weer heeft op het stressgevoel. We vroegen een aantal
personen om iedere dag hun stressgevoel te noteren. Bij deze gegevens noteerden we de gemiddelde
temperatuur, luchtvochtigheid en hoeveelheid neerslag op die dag.

1. Lees de data in. Welke kolommen bevatten NA-waarden? Verwijder de rijen met NA-waarden.
Verwijder de rijen met extreme uitschieters bij neerslag. De luchtvochtigheid mag niet hoger zijn dan 100. 
Verwijder de rijen waarbij dat zo is. Hoeveel rijen hou je nu over? 

In [4]:
from hulpfuncties import *

missing_values = ['n/a', 'na', 'nan', 'N/A', 'NA', 'NaN', 'NAN', '--', 'Missing', 'missing', 'Unknown', 'unknown']
stress = pd.read_csv("../datasets/stress.csv", sep="\t", decimal=",", na_values=missing_values)     
len(stress)


1095

In [13]:
stress.head()
stress.isnull().sum()
stress.info()
stress.head()
stress

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 0 to 1094
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   stress       1092 non-null   object 
 1   temp         1092 non-null   float64
 2   neerslag     1092 non-null   float64
 3   vochtigheid  1092 non-null   float64
dtypes: float64(3), object(1)
memory usage: 42.7+ KB


,stress,temp,neerslag,vochtigheid
0,normaal,18.3,2.5,72.0
1,licht gestresseerd,7.4,2.7,69.0
2,licht gestresseerd,12.6,2.5,76.0
3,normaal,14.1,2.3,73.0
4,licht gestresseerd,12.8,3.1,85.0
...,...,...,...,...
1090,gestresseerd,3.1,1.6,87.0
1091,licht gestresseerd,11.0,2.5,95.0
1092,licht gestresseerd,7.3,2.4,83.0
1093,licht gestresseerd,4.6,2.1,79.0


In [10]:
stress.dropna(axis=0, how='any', inplace=True)
print(len(stress))


1092


In [11]:
stress.isnull().sum()

stress         0
temp           0
neerslag       0
vochtigheid    0
dtype: int64

In [7]:
stressZenV100 = stress[~((uitschieters(stress.neerslag, mode='extreem')) | (stress['vochtigheid'] > 100))].copy()

In [8]:
len(stressZenV100)  #aantal zonder extreme neerslag en vochtigheid niet groter  dan 100

1090

2. Welk meetniveau hebben de kolommen?

stress:ordinaal temp:interval vochtigheid,neerslag:ratio

3. Wat is de gemiddelde temperatuur, luchtvochtigheid en neerslag?

In [9]:
# kan apart zoals bijvoorbeeld hieronder voor neerslag
print(f"Gemiddelde neerslag:{stressZenV100.neerslag.mean()}")
# maar via describe krijgen we alle statistieken tegelijk
stressZenV100.describe()

Gemiddelde neerslag:2.3707339449541283


,temp,neerslag,vochtigheid
count,1090.000000,1090.000000,1090.000000
mean,10.321101,2.370734,79.875229
std,5.658306,0.270014,6.220839
min,-8.500000,1.600000,60.000000
25%,6.600000,2.200000,76.000000
50%,10.400000,2.400000,80.000000
75%,14.200000,2.600000,84.000000
max,30.300000,3.300000,99.000000


4. Zoek de correlatie tussen stress en temperatuur. Welke methode gebruik je best? Welke waarde
vind je? Wat betekent deze?

In [10]:
stressZenV100.info()  # we controleren of alle variabelen het juiste datatype hebben

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 0 to 1094
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   stress       1090 non-null   object 
 1   temp         1090 non-null   float64
 2   neerslag     1090 non-null   float64
 3   vochtigheid  1090 non-null   float64
dtypes: float64(3), object(1)
memory usage: 42.6+ KB


In [11]:
# we moeten de stress variabele nog omvormen tot een categorische variabele van de ordinaal type
# we bekijken alle mogelijke waarden die deze variabele kan aannemen
stressZenV100.stress.unique()

array(['normaal', 'licht gestresseerd', 'gestresseerd',
       'help, ik verzuip!', 'zen'], dtype=object)

In [12]:
# we herschikken die hier in de juiste volgorde en maken van stress een ordinaal meetniveau
stresslevels = ['zen', 'normaal', 'licht gestresseerd', 'gestresseerd', 'help, ik verzuip!']
stressZenV100.stress = pd.Categorical(stressZenV100.stress, categories=stresslevels, ordered=True)

In [13]:
stressZenV100.info()  #controle of stress nu van type categorie is

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 0 to 1094
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   stress       1090 non-null   category
 1   temp         1090 non-null   float64 
 2   neerslag     1090 non-null   float64 
 3   vochtigheid  1090 non-null   float64 
dtypes: category(1), float64(3)
memory usage: 35.3 KB


4. Zoek de correlatie tussen stress en temperatuur. Welke methode gebruik je best? Welke waarde
vind je? Wat betekent deze?

In [14]:
# we hebben de keuze tussen spearman en Kendall omdat beide kunnen werken met ordinale data
stressZenV100.stress.corr(stressZenV100.temp, method='kendall')

0.6533314631732314

In [15]:
stressZenV100.stress.corr(stressZenV100.temp, method='spearman')

0.775593821462883

Dat er toch een aanzienlijk deel van het stress niveau positief gerelateerd is met temp. Dus hoe hoger de temperatuur hoe groter het stressniveau.


5. Zoek de correlatie tussen stress en luchtvochtigheid. Welke waarde vind je? Wat betekent deze?

In [17]:
stressZenV100.stress.corr(stressZenV100.vochtigheid, method='kendall')

-0.3220206998144224

Licht verband hoe vochtiger hoe minder het stressniveau.

6. Zoek de correlatie tussen stress en neerslag. Welke waarde vind je? Wat betekent deze?

In [18]:
stressZenV100.stress.corr(stressZenV100.neerslag, method='kendall')

0.0032704318362192096

Er is geen correlatie tussen stress en neerslag in deze dataset.

7. Welke factoren spelen dus een rol in het stressgevoel?

Enkel vochtigheid en temperatuur zijn gecorreleerd met het stressniveau in deze dataset.